# LangChain 核心模块 Agent - Self ask with search

- Google 搜索对接
    - 第三方平台：https://serpapi.com
    - LangChain API 封装：`SerpAPIWrapper`
- LangChain Agent 模块使用
    - Agent 类型：`SELF_ASK_WITH_SEARCH`
    - Agent 实例化：`initialize_agent` 方法
 
https://python.langchain.com/docs/modules/agents/agent_types/

In [45]:
import os

# 更换为自己的 Serp API KEY
os.environ["SERPAPI_API_KEY"] = "1e3c93e0753ac224098370cd71da86150c6609caba6b5aaa307e04b72a5006e1"
# print(os.environ["SERPAPI_API_KEY"])

1e3c93e0753ac224098370cd71da86150c6609caba6b5aaa307e04b72a5006e1


In [33]:
from langchain_openai import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent, AgentType, Tool

llm = OpenAI(temperature=0)

In [34]:
# 实例化查询工具
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search",
    )
]

In [35]:
# 实例化 SELF_ASK_WITH_SEARCH Agent
self_ask_with_search = initialize_agent(
    tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True, handle_parsing_errors=True
)

In [36]:
# 实际运行 Agent，查询问题（正确）
self_ask_with_search.invoke(
    "成都举办的大运会是第几届大运会？"
)



> Entering new AgentExecutor chain...
Could not parse output:  No. 
Final answer: 第28届大运会 (The 28th Universiade)
Intermediate answer: Invalid or incomplete response
Could not parse output: 
Intermediate answer: Invalid or incomplete response
So the final answer is: 第28届大运会 (The 28th Universiade)

> Finished chain.


{'input': '成都举办的大运会是第几届大运会？', 'output': '第28届大运会 (The 28th Universiade)'}

In [37]:
# 实际运行 Agent，查询问题（有歧义）
self_ask_with_search.invoke(
    "2023年大运会举办地在哪里？"
)



> Entering new AgentExecutor chain...
Could not parse output:  No.
Intermediate answer: Invalid or incomplete response
So the final answer is: Invalid or incomplete response

> Finished chain.


{'input': '2023年大运会举办地在哪里？', 'output': 'Invalid or incomplete response'}

In [38]:
# Reason-only 答案（错误）
print(llm("成都举办的大运会是第几届大运会？"))



成都举办的大运会是第31届大运会。


In [39]:
# Reason-only 答案（错误）
print(llm("2023年大运会举办地在哪里？"))



2023年大运会的举办地将在中国的重庆市。


#### 使用 GPT-4 作为大语言模型实现更优的 ReAct 范式

In [40]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)

In [41]:
self_ask_with_search_chat = initialize_agent(
    tools, chat_model, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True, handle_parsing_errors = True
)

In [42]:
# GPT-4 based ReAct 答案（正确）
self_ask_with_search_chat.run(
    "2023年大运会举办地在哪里？"
)



> Entering new AgentExecutor chain...
Yes.
Follow up: What is the 2023年大运会 in English?
Intermediate answer: 2023年夏季世界大学生运动会（英语：XXXII 2023 Summer World University Games）简称为叶卡捷琳堡大运会，是第32届夏季世界大学生运动会。
Follow up: Where is the 2023 Summer World University Games being held?
Intermediate answer: Chengdu, China
So the final answer is: 成都，中国 (Chengdu, China)

> Finished chain.


'成都，中国 (Chengdu, China)'